# Applied Data Science Capstone

This notebook will be used for the capstone project for A.

# Assignment 3: Exploring Toronto Neighborhood
<h2>Table of content</h2>
<h4>Please click on the respective submission link you would like to see</h4>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="#Submission-1">Submission 1: Building the Neighborhood data frame</a>
    <li><a href="#Submission-2">Submission 2: Adding the geolocation</a>
    <li><a href="#Submission-3">Submission 3: Exploring Toronto Neighborhoods</a>
    </ul>
    </div>

In [40]:
import pandas as pd
import numpy as np
import wget
import requests
import csv
import geocoder
import geopy

In [46]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Submission 1
In this section, I will be scraping the Wikipedia page to build a pandas dataframe to store the PostaCode, Borough and Neighborhood

In [47]:
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit

I downloaded the webpage as a html page to my local repository

In [48]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
print("Wiki Page Downloaded")

Wiki Page Downloaded


Now let's use the BeautifulSoup package to work with the html data

In [49]:
soup = BeautifulSoup(page.content, 'html.parser')
print("BeautifulSoup object created")

BeautifulSoup object created


Let's get the table and view the html tag where our data is stored

In [50]:
neigh_data = soup.find_all('table', class_='wikitable sortable')

Let's save the contents of the table into a csv

In [51]:
output_rows = []
for row in soup.find_all('table', class_='wikitable sortable'):
    columns = row.find_all('tr')
    output_row = []
    for column in columns: 
        output_row.append(column.text)
    output_rows.append(output_row)
    
with open('output.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)
  

For the sake of simplicity, I went ahead and cleaned the rows in notepad++ so I can easily read it

In [52]:
#let's open the csv file
csvfile = "C:/Users/nirin/Downloads/output.csv"
df = pd.read_csv(csvfile,
                 header=0,
                 delimiter='  ',                 
                 quotechar='"',
                 error_bad_lines=False,
                 engine='python')

Skipping line 113: Expected 3 fields in line 113, saw 4. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.


In [53]:
df.shape

(179, 3)

Let's combine the neighborhood with the same Postal Code into one

In [54]:
dfPC = df.groupby(by="Postal Code", axis=0, as_index=False, sort=True, squeeze = False)
dfPC

<ipython-input-54-31838aa6381b>:1: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  dfPC = df.groupby(by="Postal Code", axis=0, as_index=False, sort=True, squeeze = False)


In [55]:
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Shape before removing rows

In [56]:
df.shape

(179, 3)

In [57]:
df.drop(df[df['Borough'] =='Not assigned'].index, inplace = True)
print("removed rows with Not assigned Borough")

removed rows with Not assigned Borough


Shape of the data after removing 

In [58]:
df.rename(columns={"Postal Code" :"PostalCode"})

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
159,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
164,M4Y,Downtown Toronto,Church and Wellesley
167,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
168,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [59]:
df.shape

(102, 3)

#### *** end of submission 1 ***

---------------------------------------------------------------------

## Submission 2

In this section, we will be adding two additional columns for latitude and longitude of each Borough

Let's get the csv file that contains our geospatial data and save it in a dataframe

In [60]:
geodf = pd.read_csv("C:/Users/nirin/Geospatial_Coordinates.csv",                   
                 header=0,
                 delimiter=",",                 
                 error_bad_lines=False,
                 engine='python'
                   )
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combining the two data frame, our final dataframe will look like the one below:

In [65]:
dfall = df.merge(geodf)
dfall.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [64]:
dfall.shape

(102, 5)

### ***End of submission 2 ***

---------------------------------------------------------------------------------------

## Submission 3